# Team members

### Felipe Ramirez

### Paul-Antoine Croux

### Seif Labib

### Nathann Morand (296190)
Electronicien de formation.

travaille comme administrateur système accoté de l'EPFL.

Socialement peu aware.

Se débrouille en python.

# Environment
Notre environnement de travail est assez hétérogène, intégrant divers outils et plateformes.
Nous utilisons principalement PyCharm et VSCode comme IDE, avec Git intégré pour le suivi des versions pour certain et d'autre utilisent GitKraken ou l’interface web de GitHub.
Afin de limiter les problèmes de merging, nous avons défini en amont un squelette de classes pour structurer le travail de chaque membre de l'équipe.
Les noms des fonctions ainsi que leurs types de retour ont été préalablement établis, dans le but de limiter les incompatibilité et simplifier les tests.

En ce qui concerne l’environnement dans lequel évolue notre robot, nous avons imaginé un scénario où ce dernier joue le rôle d’une ambulance.
Sa mission est de se rendre sur le lieu d’un accident en empruntant des routes et en évitant les voitures (obstacles) présentes sur son trajet.
La caméra embarquée agit comme un "GPS", offrant une position absolue, mais elle est sujette à des coupures temporaires et ne détecte pas les obstacles dynamiques.
La localisation du robot et de la cible repose sur deux marqueurs ArUco sous forme de QR codes, tandis qu’un damier est utilisé pour calibrer la caméra et retrouver l’échelle en millimètres.
Enfin, le robot est contrôlé de manière asynchrone depuis Python.


# Design choice

### Architecture overview (from README.md)
Main loop :
- get the Camera position of the map and robot and the target (car crash, ciff)
- get the map layout (road line) (Camera)
- get the position from the robot encoder
- use kalmann to improve robot position
- update robot position
- use path planning to find best way (A*)
- update robot movement instruction

Fast loop :
- react to obstacle and avoid,
- follow track given by path planning

Pathplanning :
- use the occupancy matrix to compute the fastest route

ComputerVision :
- get the image dimmension + calibrate the camera.
- get the robot + goal position
- get the occupancy map

Robot :
deal with the hardware + realtime
- give estimated position
- allow the update of the estimated position
- get a list of waypoint and follow them.

sensorfusion :
- merge the data from the camera + encoder to improve the accuracy.

### Robot movement + Local obstacle avoindance -> Seif

python class for robot movement.
this is the fast loop.
It run in its own thread and handle to robot movement  + communication and local obstacle avoidance.
It will be fed a list of waypoint from the slower path planning and will follow them unless it's path is blocked and will then go around the obstacle to meet back the waypoint line. This class return the estimated position+speed from the wheel encoder.(when the position is updated from the kalman filter, keep updating it over time from the position delta from the wheel encoder so that when we read back that value later one we know where you are now estimated by the wheel encoder.)

also make the thymio blink in blue like an ambulance.

```
class RobotMovement:
    def __init__(self):
        # Initialize motors, sensors, and encoders
        self.waypoints = [np.array([0, 0, 0])] # list of np.array
        self.position = np.array([0, 0, 0])
        self.speed = np.array([0, 0, 0])

    def set_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # same format as get_waypoints
        # update the global list of waypoint to follow

    def get_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # return the global list of waypoint to follow minus the one that where reached aldready
        waypoints = [
            np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians) clockwise rotation from x axis
            np.array([200, 250, np.pi / 3]),  # x=200mm, y=250mm, direction=60° (π/3 radians)
            np.array([300, 350, np.pi / 2])   # x=300mm, y=350mm, direction=90° (π/2 radians)
        ]
        return waypoints

    def get_position(self):# Beware, this class is inside a thread you need to use a mutex !!!
        # Return estimated position from encoders
        # same format as for set_position

    def set_position(self, kalman_position):# Beware, this class is inside a thread you need to use a mutex !!!
        # Update the robot’s position based on Kalman filter results
        return np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians)

    def get_speed(self): # there is no need to set the speed from kalman, the motor know better
        # Return the speed+angular speed from encoders
        # Beware, this class is inside a thread you need to use a mutex !!!
        return np.array([10, 15, np.pi / 60]),  # x=10mm/s, y=15mm/s, direction=3°/s (π/60 radians/second)

    def update(self):# this function is called every 10 ms for you to do what you have to
                     # (avoid obstacle, go to the nearest waypoint)
        return None
```
### Computer vision and global positionning + occupancy grid -> Felipe
This class sets up a computer vision system using OpenCV, with the main goal of generating an occupancy grid and position of our robot and goal. This grid represent the environment as occupied and free area, based on images captured from a camera or loaded from a file. The grid will then be used by other modules/class in the project for tasks like navigation or pathplanning.

The VisionSystem class offers a way to capture images, allowing the use of either a real-time camera or image files, which is especially useful for testing. It uses ArUco markers to detect two key elements: the robot and the goal. Using these marker, the class can locate these objects in the environment, calculating their positions in millimeters and their orientations.

Camera calibration is included to improve the accuracy of measurements. It uses a checkerboard pattern to adjust the camera parameters and calculate the size of a pixel in millimeter. This ensures that image coordinates can be converted into real-world distances.

Finally, the main method, which generates the occupancy grid, processes the image to identify fixed obstacles in the environment. It converts the image to grayscale, applies a filter to reduce noise, and binarizes the image to create a clear map of occupied and free areas. The resulting grid will then be used by other project modules.

In summary, this class provides an occupancy grid (NxM matrix) and return the position and orientation of the robot used for path planning.
### Path Planning -> Paul-Antoine
bref tu raconte comment ça marche et pour mettre des extratide code voici la sintaxe :
```
class PathPlanner:
    #
    def get_waypoints(occupancy_grid, start, target):
        # A* algorithm to calculate path
        # Return a list of waypoints to follow, read the class of work package 1 + 2 to have the specific of the format. (robot + VisionSystem)
        start = np.array([100, 150, np.pi / 6]) # both start and target are defined as numpy array with the first component
        target = np.array([100, 150, np.pi / 6]) # being the x axis, second is y, and third is the angle relative to x axis in radian
                                                 # clockwise rotation
        waypoints = [
            np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians)
            np.array([200, 250, np.pi / 3]),  # x=200mm, y=250mm, direction=60° (π/3 radians)
            np.array([300, 350, np.pi / 2])   # x=300mm, y=350mm, direction=90° (π/2 radians)
        ]
        return waypoints
```

### sensor fusion and test script -> Nathann
I tried at first to implement a kalman filter that worked relatively well in "static" condition (linear acceleration or constant speed)
![Kalman Early Test](testData/kalmanEarlyTest.jpg)
but since the robot is changing course from time to time I did not like this solution.
instead I implemented a particule filter that was more robust to the constantly changing course.
![Particule Filter](testData/particuleFilter.jpg)
The particle filter class implement the filter itself (along each DOF of the robot so X position, Y position and theta) and offer the following function :

init: Create the particle filter with a specified number of particles and the max range (we assume that the robot will not travel outside an 10x10 meter arena nor will do more than one full turn on itself before rewinding)

Predict: Move particles by a distance obtained by mutliplying the current speed by the elapsed time and adding some noise. (the noise amplitude is determined empirically to be of the same magnitude of the noise from the sensor)

Update: this function compute the weight (higher mean more likely to reflect the true position of the robot) of each particle compared to the value returned from the sensor. (when both the camera + robot odometry are avaible, do the average between the two)

Resample: This function act as a lotery where a higher weight mean more chance of winning. it take all the "winning" particle (some particle win multiple time) and replace the population of N particle with N new particle made of the winner by the number of win of each winner. this is a zero sum game so that the particle number stay constant.

Estimate: compute the mean position of the particles to find a close approximation of the true position of the robot.

The sensor fusion class keep track of the 3 instance of the particle filter (for each DOF) and check that the value are available for each sensor (if yes, compute the average of the two sensor, if not only use the data from one sensor.

I also made some test script to help the others to verify that the code was working. They take the form of a
"if __name__ == "__main__":" in each file to test the file individually with the help of a visualisation tool.

The Visualisation library offer function function to :

update_plot : This take into argument the posisiton of the robot from the robot and the camera + the goal position and the estimated position. It also take the list of waypoints to reach the target and the occupancy grid and the raw image from openCV and make a nice chat using matplotlib :
![visualisation](testData/visualisation.jpg)

plot_robot_grid : this is similar to update_plot but more crude as it does not update the plot dynamically and is used for static test of the pathfinding algoritm and the computer vision test.

Finally I wrote the main.py and test.py. The "fast" loop freqencythat run in its own thread and control the robot is limited by the asynchronous nature of the robot as it take more than 500 ms query the robot and update its value.
The slow loop run at 0.2 Hz as matplotlib it quite slow to update the visualisation. But since the robot is quite slow this is okay.


# Demo
Pour la démo, il suffit de lancer main.py
si on désire seulement simuler en utilisant des images pré-enregistrer sans faire physiquement bouger le robot, test.py suffit.
Depuis le présent dossier, lancé les commandes suivantes :


In [ ]:
python3 -m venv .venv # setup a virtual environement to prevent library issue
source .venv/bin/activate  # or .venv\Scripts\activate on Windows
pip install -r requirements.txt # install our required library
python3 robot.py # python3 test.py

# Conclusion